In [3]:
search_for = 128
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.01768779754638672
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 330
all cases: 866251507
type: [1, 1, 1, 1, 128] 128
cases of this type: 268435456
100000 41.77981305053368
200000 80.25284823875991
300000 80.28663356529309
400000 79.97599838538513
500000 80.8292642689133
600000 87.29948453024782
700000 77.01947062504328
800000 80.8209773970061
900000 74.02525685999974
1000000 71.69121594455461
1100000 22.95795756501308
1200000 72.56064929429657
1300000 65.15778956437082
1400000 69.58309474406197
1500000 65.16157383085124
1600000 73.73283916238094
1700000 66.38320943230717
1800000 70.21879248839186
1900000 66.09340680646095
2000000 70.62852870977233
2100000 52.090548502133025
2200000 35.835458141377494
2300000 69.21766217804415
2400000 64.39000199184497
2500000 68.81290773243512
2600000 64.05309253884383
2700000 71.21015764998053
2800000 63.432045302961384
2900000 63.69635675516822
3000000 67.53087293537033


30100000 72.67183029098311
30200000 73.86819761393518
30300000 66.43905099046458
30400000 59.633963716481226
30500000 39.97228775469567
30600000 69.07985490607848
30700000 75.02594765132037
30800000 69.9386524794341
30900000 74.71790785113978
31000000 72.90162712334579
31100000 75.63080427415676
31200000 73.50812127718225
31300000 36.19558454814665
31400000 76.15894906933947
31500000 78.78542051506977
31600000 76.13881811638393
31700000 62.54390916900333
31800000 46.12996255648952
31900000 73.39723167353986
32000000 77.70739444722821
32100000 75.56323786072298
32200000 69.8393281497178
32300000 73.78563485119636
32400000 72.4654824689327
32500000 56.712250767734524
32600000 43.002309197863184
32700000 69.72398695735035
32800000 71.17777864698284
32900000 70.80114256528188
33000000 74.70942404619929
33100000 73.27228424987484
33200000 79.56726687020053
33300000 65.60045730500337
33400000 49.773409860287124
33500000 75.74023942714447
33600000 78.6450440047987
33700000 75.93691409928437
3

60600000 77.6623594184682
60700000 76.658968778373
60800000 78.77254968469501
60900000 77.68709400312203
61000000 82.42769254834563
61100000 77.06340393441432
61200000 76.19007942010785
61300000 54.98875460207302
61400000 49.306887863121524
61500000 70.86275426621299
61600000 67.33031560997638
61700000 70.24042734419145
61800000 69.24481128834383
61900000 32.66127304710753
62000000 64.15991041801394
62100000 71.90405528543992
62200000 69.08255702617807
62300000 67.11439612011576
62400000 74.5589247034692
62500000 33.418090682666794
62600000 75.47191580524886
62700000 76.17833210877787
62800000 77.10634359521907
62900000 78.84527373959811
63000000 85.37977739433268
63100000 76.07422672809405
63200000 73.69424816161288
63300000 72.73923143563756
63400000 59.92632143445574
63500000 44.90188591244511
63600000 67.3037021628015
63700000 71.03498144706786
63800000 67.46784180184291
63900000 66.96443517725932
64000000 28.193620787363145
64100000 67.17658390229302
64200000 69.6608224385001
6430

91000000 51.68964320773067
91100000 63.24020741648898
91200000 62.7546048628097
91300000 25.6918462999601
91400000 66.02553988355933
91500000 65.01793047818401
91600000 39.3042077851808
91700000 60.386405209275715
91800000 63.49751469210089
91900000 68.59941697569074
92000000 68.2995343149131
92100000 76.17752297598324
92200000 73.37931754606498
92300000 73.02127460619029
92400000 71.70136340792021
92500000 69.88722107124862
92600000 71.18051692876286
92700000 65.79684220505474
92800000 65.08463996682987
92900000 62.769680810893306
93000000 48.46495818843641
93100000 43.27246794549582
93200000 66.63561471321252
93300000 59.160695726152824
93400000 26.13659979628523
93500000 64.32451897420299
93600000 62.362108634011186
93700000 30.976167027472282
93800000 67.7072247404074
93900000 62.55024822541536
94000000 63.14444866650234
94100000 69.19586327729726
94200000 72.99410170887042
94300000 69.57401591293659
94400000 71.50296943830384
94500000 75.59054358084889
94600000 73.57751233553235
9

120600000 28.822500435844614
120700000 32.59414293021693
120800000 47.14768687301811
120900000 62.09754775279265
121000000 63.33074535362701
121100000 64.38037976244776
121200000 64.70460547779304
121300000 61.5822109063947
121400000 67.66993619627578
121500000 63.39691034023745
121600000 70.05572028028686
121700000 69.75866874242551
121800000 61.8203701036306
121900000 62.06280447130813
122000000 62.420799986500384
122100000 62.70709102276019
122200000 62.227594018907
122300000 60.63233045270685
122400000 60.55479595794132
122500000 61.0532659897035
122600000 40.159759246080135
122700000 20.13503461833211
122800000 23.745378251278012
122900000 50.09843449936972
123000000 64.11562477467078
123100000 62.51108468867277
123200000 60.4523953989867
123300000 63.08746314164516
123400000 65.05562207162346
123500000 61.6670922980229
123600000 60.742066457586745
123700000 65.91872427719998
123800000 70.58051943453046
123900000 65.17720724033585
124000000 65.38977471181786
124100000 65.254457904

...

799300000 0.4699021606018384
best so far: 0
type: [1, 2, 8, 2, 4] 128
cases of this type: 262144
799400000 0.4773968409494957
799500000 0.21595958461478554
best so far: 0
type: [1, 2, 8, 4, 2] 128
cases of this type: 131072
799600000 0.21376772176321746
799700000 0.19148100297770765
best so far: 0
type: [1, 2, 8, 8, 1] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 1, 4] 128
cases of this type: 131072
799800000 0.19349130733665298
799900000 0.21503912019448082
best so far: 0
type: [1, 2, 16, 2, 2] 128
cases of this type: 65536
best so far: 0
type: [1, 2, 16, 4, 1] 128
cases of this type: 32768
800000000 0.19285726724902852
best so far: 0
type: [1, 2, 32, 1, 2] 128
cases of this type: 32768
best so far: 0
type: [1, 2, 32, 2, 1] 128
cases of this type: 16384
best so far: 0
type: [1, 2, 64, 1, 1] 128
cases of this type: 8192
best so far: 0
type: [1, 4, 1, 1, 32] 128
cases of this type: 4194304
800100000 0.40998905055509144
800200000 1.0881532140557368
800300000 0.8848810381

812600000 0.8796939500885633
812700000 0.8395999860887738
812800000 0.8636006669870615
812900000 0.8568277108586584
813000000 0.8160986299426384
813100000 0.7520872564158082
813200000 0.778802033134687
813300000 0.8794514768406676
813400000 0.7920660301949481
813500000 0.764039737275992
813600000 0.7104279069678386
813700000 0.7621326240825494
813800000 0.8666453913934191
813900000 0.7149675300712467
814000000 0.7043202271701515
814100000 0.7768706815496902
814200000 0.7820866358621384
814300000 0.743807474720633
814400000 0.7136116476624085
814500000 0.7062199473174062
814600000 0.7778319612054289
814700000 0.828895108994853
814800000 0.724728589845348
814900000 0.7993077668656767
815000000 0.6966919184574976
815100000 0.6874418197108162
815200000 0.7652690850456179
815300000 0.6956257165721542
815400000 0.7832542370195938
815500000 0.697221903920716
815600000 0.6396868224362042
815700000 0.7283957778085238
815800000 0.6163702273009479
815900000 0.7444733923787878
816000000 0.74785665

840200000 0.14518871949621506
840300000 0.2299195579215341
840400000 0.22481264325836497
840500000 0.14430775015935635
840600000 0.22732893946758972
840700000 0.17033990081738629
840800000 0.1865993026389122
840900000 0.2260486399223758
841000000 0.12794159599485397
841100000 0.232257891769437
841200000 0.1976373922298299
841300000 0.15669375392804744
841400000 0.22135613094928264
841500000 0.15387153490107855
841600000 0.2047740933467865
841700000 0.21822310096788472
841800000 0.1328136817546904
best so far: 0
type: [2, 1, 1, 8, 8] 128
cases of this type: 2097152
841900000 0.23567801850724815
842000000 0.182437122455181
842100000 0.14653859330219163
842200000 0.2164712253339966
842300000 0.1468467843277865
842400000 0.17126345628819795
842500000 0.19239893398241995
842600000 0.13698913449857897
842700000 0.1925522451044705
842800000 0.1565291252609909
842900000 0.1481643655008581
843000000 0.19352526182963717
843100000 0.13544914593285123
843200000 0.17114573636734218
843300000 0.1655

best so far: 0
type: [2, 2, 1, 32, 1] 128
cases of this type: 65536
860800000 0.01539728620019555
best so far: 0
type: [2, 2, 2, 1, 16] 128
cases of this type: 524288
860900000 0.028829416563682427
861000000 0.037401730505970454
861100000 0.025578665276191632
861200000 0.016266009119211302
861300000 0.032957924243837596
best so far: 0
type: [2, 2, 2, 2, 8] 128
cases of this type: 262144
861400000 0.03405328952906529
861500000 0.028139212463761037
861600000 0.015206143835523392
best so far: 0
type: [2, 2, 2, 4, 4] 128
cases of this type: 131072
861700000 0.025287328608495
best so far: 0
type: [2, 2, 2, 8, 2] 128
cases of this type: 65536
861800000 0.018979685716042253
best so far: 0
type: [2, 2, 2, 16, 1] 128
cases of this type: 32768
best so far: 0
type: [2, 2, 4, 1, 8] 128
cases of this type: 131072
861900000 0.019432408917007843
best so far: 0
type: [2, 2, 4, 2, 4] 128
cases of this type: 65536
862000000 0.01747866848582228
best so far: 0
type: [2, 2, 4, 4, 2] 128
cases of this type:

866100000 0.0006075447502387895
best so far: 0
type: [8, 1, 1, 2, 8] 128
cases of this type: 32768
best so far: 0
type: [8, 1, 1, 4, 4] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 1, 8, 2] 128
cases of this type: 8192
best so far: 0
type: [8, 1, 1, 16, 1] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 1, 8] 128
cases of this type: 16384
best so far: 0
type: [8, 1, 2, 2, 4] 128
cases of this type: 8192
866200000 0.00019855630949735641
best so far: 0
type: [8, 1, 2, 4, 2] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 2, 8, 1] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 1, 4] 128
cases of this type: 4096
best so far: 0
type: [8, 1, 4, 2, 2] 128
cases of this type: 2048
best so far: 0
type: [8, 1, 4, 4, 1] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 1, 2] 128
cases of this type: 1024
best so far: 0
type: [8, 1, 8, 2, 1] 128
cases of this type: 512
best so far: 0
type: [8, 1, 16, 1, 1] 128
cases of this type: 256
best so 